In [1]:
import copy
import datetime
import pandas as pd
import numpy as np
import scrapbook as sb
from dateutil import tz

import systemlink.clients.nitestmonitor as testmon

In [2]:
results_filter = 'startedAt <= "2021-07-01T20:00:00Z" && startedAt >= "2021-07-01T00:00:00Z" && programName = "CTS_eStick_Manual.seq" && properties["Location"].Contains("AR3") && workspaceName = "EPL Manufacturing"'
products_filter = ''
group_by = 'Day'

In [3]:
groups_map = {
    'Day': 'started_at',
    'System': 'system_id',
    'Test Program': 'program_name',
    'Operator': 'operator',
    'Part Number': 'part_number',
    'Workspace': 'workspace'
}

grouping = groups_map[group_by]
display(grouping)

'started_at'

In [4]:
results_api = testmon.ResultsApi()

In [5]:
results_query = testmon.ResultsAdvancedQuery(
    results_filter, product_filter=products_filter, order_by=testmon.ResultField.STARTED_AT
)

results = []

response = await results_api.query_results_v2(post_body=results_query)

while response.continuation_token:
    results = results + response.results
    results_query.continuation_token = response.continuation_token
    response = await results_api.query_results_v2(post_body=results_query)
    
results_list = []

for result in results:
    results_list.append(result.to_dict())
    

In [6]:
group_names = []
for result in results_list:
    if grouping in result:
        group_names.append(result[grouping])


In [7]:
df_results = pd.DataFrame.from_records(results_list)

In [9]:
df_results = df_results.join(df_results['status'].apply(pd.Series))

In [10]:
del df_results['status']
del df_results['status_type_summary']

In [11]:
df_results

,started_at,updated_at,program_name,id,system_id,host_name,operator,part_number,serial_number,total_time_in_seconds,keywords,properties,file_ids,workspace,status_type,status_name
0,2021-07-01 04:21:11.882406+00:00,2021-07-01 04:22:26.237529+00:00,CTS_eStick_Manual.seq,60dd42f0ca744a307c1b22e2,P-2100--SN-Default_string--MAC-2C-94-64-03-C9-68,EQ1000757-1,MPAKI,C00311,16824025,14.337192,[],"{'DUT Handler FW': '1.08', 'DUT Handler Port':...",[60dd42c9ca744a391806b5f6],e87cbce3-c5f9-424d-aaa4-c2209d9233b6,FAILED,None
1,2021-07-01 04:21:14.501051+00:00,2021-07-01 04:22:31.841608+00:00,CTS_eStick_Manual.seq,60dd42f1ca744a307c1b22f0,P-2100--SN-Default_string--MAC-2C-94-64-03-C9-68,EQ1000757-1,MPAKI,C00311,16824026,13.940914,[],"{'DUT Handler FW': '1.08', 'DUT Handler Port':...",[60dd42caca744a391806b5f7],e87cbce3-c5f9-424d-aaa4-c2209d9233b6,FAILED,None
2,2021-07-01 04:21:17.009378+00:00,2021-07-01 04:22:37.343052+00:00,CTS_eStick_Manual.seq,60dd42f1ca744a307c1b22fc,P-2100--SN-Default_string--MAC-2C-94-64-03-C9-68,EQ1000757-1,MPAKI,C00311,16824027,13.957605,[],"{'DUT Handler FW': '1.08', 'DUT Handler Port':...",[60dd42cdca744a391806b5f8],e87cbce3-c5f9-424d-aaa4-c2209d9233b6,FAILED,None
3,2021-07-01 04:21:19.219336+00:00,2021-07-01 04:22:42.714313+00:00,CTS_eStick_Manual.seq,60dd42f2ca744a307c1b2309,P-2100--SN-Default_string--MAC-2C-94-64-03-C9-68,EQ1000757-1,MPAKI,C00311,16824028,13.908029,[],"{'DUT Handler FW': '1.08', 'DUT Handler Port':...",[60dd42cfca744a391806b5f9],e87cbce3-c5f9-424d-aaa4-c2209d9233b6,FAILED,None
4,2021-07-01 04:21:21.292532+00:00,2021-07-01 04:22:47.555789+00:00,CTS_eStick_Manual.seq,60dd42f2ca744a307c1b2317,P-2100--SN-Default_string--MAC-2C-94-64-03-C9-68,EQ1000757-1,MPAKI,C00311,16824029,13.947179,[],"{'DUT Handler FW': '1.08', 'DUT Handler Port':...",[60dd42d1ca744a391806b5fa],e87cbce3-c5f9-424d-aaa4-c2209d9233b6,FAILED,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
869,2021-07-01 07:38:15.473007+00:00,2021-07-01 07:40:38.749006+00:00,CTS_eStick_Manual.seq,60dd7146ca744a307c1e806b,P-2100--SN-Default_string--MAC-2C-94-64-03-C9-68,EQ1000757-1,SJAMESON,M1000843,16824433,12.258310,[],"{'DUT Handler FW': '1.08', 'DUT Handler Port':...",[60dd70f7ca744a391806b968],e87cbce3-c5f9-424d-aaa4-c2209d9233b6,PASSED,None
870,2021-07-01 07:38:16.313759+00:00,2021-07-01 07:40:38.802003+00:00,CTS_eStick_Manual.seq,60dd714dca744a307c1e80d8,P-2100--SN-Default_string--MAC-2C-94-64-03-C9-68,EQ1000757-1,SJAMESON,M1000843,16824434,12.197281,[],"{'DUT Handler FW': '1.08', 'DUT Handler Port':...",[60dd70f8ca744a391806b969],e87cbce3-c5f9-424d-aaa4-c2209d9233b6,PASSED,None
871,2021-07-01 07:38:50.197268+00:00,2021-07-01 07:41:00.475005+00:00,CTS_eStick_Manual.seq,60dd717fca744a307c1e8473,P-2100--SN-Default_string--MAC-2C-94-64-03-C9-68,EQ1000757-1,SJAMESON,M1000843,16824428,11.506059,[],"{'DUT Handler FW': '1.08', 'DUT Handler Port':...",[60dd7119ca744a391806b96a],e87cbce3-c5f9-424d-aaa4-c2209d9233b6,FAILED,None
872,2021-07-01 07:38:51.279397+00:00,2021-07-01 07:41:01.080017+00:00,CTS_eStick_Manual.seq,60dd717fca744a307c1e847f,P-2100--SN-Default_string--MAC-2C-94-64-03-C9-68,EQ1000757-1,SJAMESON,M1000843,16824435,12.085654,[],"{'DUT Handler FW': '1.08', 'DUT Handler Port':...",[60dd711aca744a391806b96b],e87cbce3-c5f9-424d-aaa4-c2209d9233b6,PASSED,None


In [12]:
df_results['Job Number'] = df_results['properties'].apply(lambda t: t['Job Number'] if 'Job Number' in t else '')
df_results['Lot Number'] = df_results['properties'].apply(lambda t: t['Lot Number'] if 'Lot Number' in t else '')
properties = pd.json_normalize(df_results['properties'])
df_results = pd.concat([df_results.drop(['properties'], axis=1), properties], axis=1)
pd.set_option('display.max_columns', 50)

In [13]:
to_zone = tz.tzlocal()
utc = df_results['started_at']

In [14]:
def astimezone(x):
    return x.astimezone(to_zone)

In [15]:
central = utc.apply(astimezone)
df_results = pd.concat([df_results.drop(['started_at'], axis=1), central], axis=1)

In [16]:
utc = df_results['updated_at']
central = utc.apply(astimezone)
df_results = pd.concat([df_results.drop(['updated_at'], axis=1), central], axis=1)

In [17]:
df_results.head(2)

,program_name,id,system_id,host_name,operator,part_number,serial_number,total_time_in_seconds,keywords,file_ids,workspace,status_type,status_name,Job Number,Lot Number,DUT Handler FW,DUT Handler Port,DUT Handler SN,EQ Number,Expiry,Firmware Version,GUI Software Name,GUI Software Version,Job Number,Location,Lot Number,Product,Programming Lot,TS SN Current,TS SN End,TS SN Start,Team Lead,Test Limit,Test Mode,Test Sequence,Test System Type,User,nitmProcessModel,nitmSource,nitmTestSocketCount,nitmTestSocketIndex,nitmTestStandStartTime,started_at,updated_at
0,CTS_eStick_Manual.seq,60dd42f0ca744a307c1b22e2,P-2100--SN-Default_string--MAC-2C-94-64-03-C9-68,EQ1000757-1,MPAKI,C00311,16824025,14.337192,[],[60dd42c9ca744a391806b5f6],e87cbce3-c5f9-424d-aaa4-c2209d9233b6,FAILED,None,6171-R&D-00000036,869535,1.08,1.0,13,EQ1000757-1,2021-12-31,08.03 v03 R03,CTS Test Software,1.0.0.76,6171-R&D-00000036,EPL-AR3-Line1,869535,C00311,21003-010,16824025,33554431,16777216,MPAKI,D1028237_A_Test_Limits.csv,Manufacturing,D1028236_A_TSM.seq,eStick Manual,DVERHOEVEN,Parallel,niteststand,16,0,2021-07-01T04:21:11.7330000Z,2021-07-01 14:21:11.882406+10:00,2021-07-01 14:22:26.237529+10:00
1,CTS_eStick_Manual.seq,60dd42f1ca744a307c1b22f0,P-2100--SN-Default_string--MAC-2C-94-64-03-C9-68,EQ1000757-1,MPAKI,C00311,16824026,13.940914,[],[60dd42caca744a391806b5f7],e87cbce3-c5f9-424d-aaa4-c2209d9233b6,FAILED,None,6171-R&D-00000036,869535,1.08,2.0,13,EQ1000757-1,2021-12-31,08.03 v03 R03,CTS Test Software,1.0.0.76,6171-R&D-00000036,EPL-AR3-Line1,869535,C00311,21003-010,16824026,33554431,16777216,MPAKI,D1028237_A_Test_Limits.csv,Manufacturing,D1028236_A_TSM.seq,eStick Manual,DVERHOEVEN,Parallel,niteststand,16,1,2021-07-01T04:21:14.4800000Z,2021-07-01 14:21:14.501051+10:00,2021-07-01 14:22:31.841608+10:00


In [18]:
df_results['started_at'] = df_results['started_at'].apply(lambda t: t.tz_localize(None))
df_results['updated_at'] = df_results['updated_at'].apply(lambda t: t.tz_localize(None))

In [19]:
df_dict = {
    'columns': pd.io.json.build_table_schema(df_results, index=False)['fields'],
    'values': df_results.values.tolist(),
}

In [20]:
results_graph = {
    'type': 'data_frame',
    'id': 'results_graph',
    'data': df_dict
}

result = [results_graph]

